##### Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import nltk
nltk.download('all')
import pickle
import gensim.downloader as api
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
import os
import re
pd.set_option('display.max_columns', None)

##### Remove Stop Words
>- A stop word is a word that is not important to the meaning of the text. For example, the word "the" is a stop word.
>- We remove these stop words because words like "the" and "and" can't help us identify a user

In [3]:
nltk.download()
stopWords = nltk.corpus.stopwords.words('english')

def removeStopWords(text):
    return [w for w in text if w not in stopWords]

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


##### Remove Symbols
>- We remove symbols because they are not important to the meaning of the text.

In [ ]:
def removeSymbols(words):
    return [w for w in words if w.isalnum()]

##### Lemmatization
>- Lemmatization is the process of finding the base form of a word.
>- For example, "sing" is the base form of "singing".

In [ ]:
wordnet_lemmatizer = WordNetLemmatizer()
def lemmatizeStemming(text):
    return wordnet_lemmatizer.lemmatize(text)

In [2]:
# This month switch is used to convert the string month given by google to a number
monthSwitch = {
    'Jan': 1,
    'Feb': 2,
    'Mar': 3,
    'Apr': 4,
    'May': 5,
    'Jun': 6,
    'Jul': 7,
    'Aug': 8,
    'Sep': 9,
    'Oct': 10,
    'Nov': 11,
    'Dec': 12
}

##### Clean user searches
>- This function takes a path to a user's data and saves a clean version of the data as a pickle.

In [5]:
def createPkl(userPath):
    search = open(userPath + '/' + 'My Activity' + '/' + 'Search' + '/' + 'MyActivity.html', encoding='utf-8')
    soup = BeautifulSoup(search, 'html.parser')
    text = soup.get_text()
    searchSplit = text.split('Searched for')
    searchSplit = searchSplit[1:]
    df = pd.DataFrame(columns=['Search', 'Day', 'Month', 'Year', 'Time'])
    
    # Loop through each search
    for s in searchSplit:
        try: 
            temp = s.split(',')
            if len(temp) == 1:
                continue
            # Get the day, month, year and query of the search
            day = temp[0].split(' ')[-3]
            month = temp[0].split(' ')[-2]
            year = temp[0].split(' ')[-1]
            searchQuery = re.findall('[a-zA-Z][^0-9A-Z]*', temp[0])[0]
            time = temp[1].split(" ")[1]
            if isinstance(int(day), int) and isinstance(monthSwitch[month], int) and isinstance(int(year), int):
                # create a row in the data frame that includes our new query
                df = df.append({'Search': searchQuery, 'Day': int(day), 'Month': monthSwitch[month], 'Year': int(year), 'Time': time}, ignore_index=True)
        except Exception:
            continue
    try:
        df[['Day', 'Month', 'Year']] = df[['Day', 'Month', 'Year']].astype(int)
    except Exception:
        pass

    # Now we perform data clearning, removing numbers, converting to lower case, removing stop words, removing symbols and applyinh lemmatization. Then we save this as a pickle to access later in model.ipynb.
    df.dropna(inplace=True)
    df['Search'] = df['Search'].str.replace('[0-9]', '')
    df['Search'] = df['Search'].str.lower()
    df['Search'] = df['Search'].apply(removeStopWords)
    df['Search'] = df['Search'].apply(removeSymbols)
    df['Search'] = df['Search'].apply(lemmatizeStemming)
    df.to_pickle(userPath + '/' + 'search.pkl')

##### Run
>- Iterate through all of the data within the Data Folder and call createPkl() on each user.

In [6]:
usersPath = '../Data'
for user in os.listdir(usersPath):
    createPkl(usersPath+'/'+user)

C:\Users\Josh\AppData\Local\Temp/ipykernel_8896/2871592760.py:33: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Search'] = df['Search'].str.replace('[0-9]', '')
